# Load Libraries

In [1]:
import numpy as np
import pandas as pd
import six

from random import randint

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

from sklearn.model_selection import train_test_split

from skimage.transform import resize
from keras.preprocessing.image import load_img

from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout,BatchNormalization
from keras.layers import Conv2D, Concatenate, MaxPooling2D
from keras.layers import UpSampling2D, Dropout, BatchNormalization
from tqdm import tqdm_notebook
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.utils import conv_utils
from keras.utils.data_utils import get_file
from keras.engine.topology import get_source_inputs
from keras.engine import InputSpec
from keras import backend as K
from keras.regularizers import l2

from keras.engine.topology import Input
from keras.engine.training import Model
from keras.layers.convolutional import Conv2D, UpSampling2D, Conv2DTranspose
from keras.layers.core import Activation, SpatialDropout2D
from keras.layers.merge import concatenate,add
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add

# Personal imports
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import load_model
from keras.losses import mean_absolute_error
import tensorflow as tf
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
train_path = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/augument_data/training_images'
npy_img_dir = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)\\augument_data\\training_images_batches'
architecture_plot_save_dir = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/'

img_width, img_height = 299., 299.
rx = 2.1404
ry = 1.61

base_dir = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/'
epochs = 1000

# Save the model according to the conditions  
chkpt_dir = "X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/resnet34_chkpt.h5" # Change for every new training session 

# Params and helpers

In [3]:
img_size_ori = 101
img_size_target = 128

def upsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_target, img_size_target), mode='constant', preserve_range=True)
    #res = np.zeros((img_size_target, img_size_target), dtype=img.dtype)
    #res[:img_size_ori, :img_size_ori] = img
    #return res
    
def downsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_ori, img_size_ori), mode='constant', preserve_range=True)
    #return img[:img_size_ori, :img_size_ori]

# Build U-Net Model

In [4]:
# https://github.com/qubvel/segmentation_models/blob/master/segmentation_models/unet/models.py

def handle_block_names_old(stage):
    conv_name = 'decoder_stage{}_conv'.format(stage)
    bn_name = 'decoder_stage{}_bn'.format(stage)
    relu_name = 'decoder_stage{}_relu'.format(stage)
    up_name = 'decoder_stage{}_upsample'.format(stage)
    return conv_name, bn_name, relu_name, up_name


def Upsample2D_block(filters, stage, kernel_size=(3,3), upsample_rate=(2,2),
                     batchnorm=False, skip=None):

    def layer(input_tensor):

        conv_name, bn_name, relu_name, up_name = handle_block_names_old(stage)

        x = UpSampling2D(size=upsample_rate, name=up_name)(input_tensor)

        if skip is not None:
            x = Concatenate()([x, skip])

        x = Conv2D(filters, kernel_size, padding='same', name=conv_name+'1')(x)
        if batchnorm:
            x = BatchNormalization(name=bn_name+'1')(x)
        x = Activation('relu', name=relu_name+'1')(x)

        x = Conv2D(filters, kernel_size, padding='same', name=conv_name+'2')(x)
        if batchnorm:
            x = BatchNormalization(name=bn_name+'2')(x)
        x = Activation('relu', name=relu_name+'2')(x)

        return x
    return layer


def Transpose2D_block(filters, stage, kernel_size=(3,3), upsample_rate=(2,2),
                      transpose_kernel_size=(4,4), batchnorm=False, skip=None):

    def layer(input_tensor):

        conv_name, bn_name, relu_name, up_name = handle_block_names_old(stage)

        x = Conv2DTranspose(filters, transpose_kernel_size, strides=upsample_rate,
                            padding='same', name=up_name)(input_tensor)
        if batchnorm:
            x = BatchNormalization(name=bn_name+'1')(x)
        x = Activation('relu', name=relu_name+'1')(x)

        if skip is not None:
            x = Concatenate()([x, skip])

        x = Conv2D(filters, kernel_size, padding='same', name=conv_name+'2')(x)
        if batchnorm:
            x = BatchNormalization(name=bn_name+'2')(x)
        x = Activation('relu', name=relu_name+'2')(x)

        return x
    return layer

In [5]:
# default parameters for convolution and batchnorm layers of ResNet models
# parameters are obtained from MXNet converted model

def get_conv_params(**params):
    default_conv_params = {
        'kernel_initializer': 'glorot_uniform',
        'use_bias': False,
        'padding': 'valid',
    }
    default_conv_params.update(params)
    return default_conv_params

def get_bn_params(**params):
    default_bn_params = {
        'axis': 3,
        'momentum': 0.99,
        'epsilon': 2e-5,
        'center': True,
        'scale': True,
    }
    default_bn_params.update(params)
    return default_bn_params

In [6]:
def handle_block_names(stage, block):
    name_base = 'stage{}_unit{}_'.format(stage + 1, block + 1)
    conv_name = name_base + 'conv'
    bn_name = name_base + 'bn'
    relu_name = name_base + 'relu'
    sc_name = name_base + 'sc'
    return conv_name, bn_name, relu_name, sc_name


def basic_identity_block(filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """

    def layer(input_tensor):
        conv_params = get_conv_params()
        bn_params = get_bn_params()
        conv_name, bn_name, relu_name, sc_name = handle_block_names(stage, block)

        x = BatchNormalization(name=bn_name + '1', **bn_params)(input_tensor)
        x = Activation('relu', name=relu_name + '1')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), name=conv_name + '1', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '2', **bn_params)(x)
        x = Activation('relu', name=relu_name + '2')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), name=conv_name + '2', **conv_params)(x)

        x = Add()([x, input_tensor])
        return x

    return layer


def basic_conv_block(filters, stage, block, strides=(2, 2)):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """

    def layer(input_tensor):
        conv_params = get_conv_params()
        bn_params = get_bn_params()
        conv_name, bn_name, relu_name, sc_name = handle_block_names(stage, block)

        x = BatchNormalization(name=bn_name + '1', **bn_params)(input_tensor)
        x = Activation('relu', name=relu_name + '1')(x)
        shortcut = x
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), strides=strides, name=conv_name + '1', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '2', **bn_params)(x)
        x = Activation('relu', name=relu_name + '2')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), name=conv_name + '2', **conv_params)(x)

        shortcut = Conv2D(filters, (1, 1), name=sc_name, strides=strides, **conv_params)(shortcut)
        x = Add()([x, shortcut])
        return x

    return layer


def usual_conv_block(filters, stage, block, strides=(2, 2)):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """

    def layer(input_tensor):
        conv_params = get_conv_params()
        bn_params = get_bn_params()
        conv_name, bn_name, relu_name, sc_name = handle_block_names(stage, block)

        x = BatchNormalization(name=bn_name + '1', **bn_params)(input_tensor)
        x = Activation('relu', name=relu_name + '1')(x)
        shortcut = x
        x = Conv2D(filters, (1, 1), name=conv_name + '1', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '2', **bn_params)(x)
        x = Activation('relu', name=relu_name + '2')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), strides=strides, name=conv_name + '2', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '3', **bn_params)(x)
        x = Activation('relu', name=relu_name + '3')(x)
        x = Conv2D(filters*4, (1, 1), name=conv_name + '3', **conv_params)(x)

        shortcut = Conv2D(filters*4, (1, 1), name=sc_name, strides=strides, **conv_params)(shortcut)
        x = Add()([x, shortcut])
        return x

    return layer


def usual_identity_block(filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """

    def layer(input_tensor):
        conv_params = get_conv_params()
        bn_params = get_bn_params()
        conv_name, bn_name, relu_name, sc_name = handle_block_names(stage, block)

        x = BatchNormalization(name=bn_name + '1', **bn_params)(input_tensor)
        x = Activation('relu', name=relu_name + '1')(x)
        x = Conv2D(filters, (1, 1), name=conv_name + '1', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '2', **bn_params)(x)
        x = Activation('relu', name=relu_name + '2')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), name=conv_name + '2', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '3', **bn_params)(x)
        x = Activation('relu', name=relu_name + '3')(x)
        x = Conv2D(filters*4, (1, 1), name=conv_name + '3', **conv_params)(x)

        x = Add()([x, input_tensor])
        return x

    return layer

In [7]:
def build_unet(backbone, classes, last_block_filters, skip_layers,
               n_upsample_blocks=5, upsample_rates=(2,2,2,2,2),
               block_type='upsampling', activation='sigmoid',
               **kwargs):

    input = backbone.input
    x = backbone.output

    if block_type == 'transpose':
        up_block = Transpose2D_block
    else:
        up_block = Upsample2D_block

    # convert layer names to indices
    skip_layers = ([get_layer_number(backbone, l) if isinstance(l, str) else l
                    for l in skip_layers])
    for i in range(n_upsample_blocks):

        # check if there is a skip connection
        if i < len(skip_layers):
#             print(backbone.layers[skip_layers[i]])
#             print(backbone.layers[skip_layers[i]].output)
            skip = backbone.layers[skip_layers[i]].output
        else:
            skip = None

        up_size = (upsample_rates[i], upsample_rates[i])
        filters = last_block_filters * 2**(n_upsample_blocks-(i+1))

        x = up_block(filters, i, upsample_rate=up_size, skip=skip, **kwargs)(x)

    if classes < 2:
        activation = 'sigmoid'

    x = Conv2D(classes, (3,3), padding='same', name='final_conv')(x)
    x = Activation(activation, name=activation)(x)

    model = Model(input, x)

    return model

# ResNet 34

In [8]:
import keras.backend as K
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import GlobalAveragePooling2D
from keras.layers import ZeroPadding2D
from keras.layers import Dense
from keras.models import Model
from keras.engine import get_source_inputs

import keras
from distutils.version import StrictVersion

if StrictVersion(keras.__version__) < StrictVersion('2.2.0'):
    from keras.applications.imagenet_utils import _obtain_input_shape
else:
    from keras_applications.imagenet_utils import _obtain_input_shape

def build_resnet(
     repetitions=(2, 2, 2, 2),
     include_top=True,
     input_tensor=None,
     input_shape=None,
     classes=1000,
     block_type='usual'):

    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=101,
                                      data_format='channels_last',
                                      require_flatten=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape, name='data')
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    
    # get parameters for model layers
    no_scale_bn_params = get_bn_params(scale=False)
    bn_params = get_bn_params()
    conv_params = get_conv_params()
    init_filters = 64

    if block_type == 'basic':
        conv_block = basic_conv_block
        identity_block = basic_identity_block
    else:
        conv_block = usual_conv_block
        identity_block = usual_identity_block
    
    # resnet bottom
    x = BatchNormalization(name='bn_data', **no_scale_bn_params)(img_input)
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(init_filters, (7, 7), strides=(2, 2), name='conv0', **conv_params)(x)
    x = BatchNormalization(name='bn0', **bn_params)(x)
    x = Activation('relu', name='relu0')(x)
    x = ZeroPadding2D(padding=(1, 1))(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='valid', name='pooling0')(x)
    
    # resnet body
    for stage, rep in enumerate(repetitions):
        for block in range(rep):
            
            filters = init_filters * (2**stage)
            
            # first block of first stage without strides because we have maxpooling before
            if block == 0 and stage == 0:
                x = conv_block(filters, stage, block, strides=(1, 1))(x)
                
            elif block == 0:
                x = conv_block(filters, stage, block, strides=(2, 2))(x)
                
            else:
                x = identity_block(filters, stage, block)(x)
                
    x = BatchNormalization(name='bn1', **bn_params)(x)
    x = Activation('relu', name='relu1')(x)

    # resnet top
    if include_top:
        x = GlobalAveragePooling2D(name='pool1')(x)
        x = Dense(classes, name='fc1')(x)
        x = Activation('softmax', name='softmax')(x)

    # Ensure that the model takes into account any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
        
    # Create model.
    model = Model(inputs, x)

    return model

In [9]:
weights_collection = [
    # ResNet34
    {
        'model': 'resnet34',
        'dataset': 'imagenet',
        'classes': 1000,
        'include_top': True,
        'url': 'https://github.com/qubvel/classification_models/releases/download/0.0.1/resnet34_imagenet_1000.h5',
        'name': 'resnet34_imagenet_1000.h5',
        'md5': '2ac8277412f65e5d047f255bcbd10383',
    },

    {
        'model': 'resnet34',
        'dataset': 'imagenet',
        'classes': 1000,
        'include_top': False,
        'url': 'https://github.com/qubvel/classification_models/releases/download/0.0.1/resnet34_imagenet_1000_no_top.h5',
        'name': 'resnet34_imagenet_1000_no_top.h5',
        'md5': '8caaa0ad39d927cb8ba5385bf945d582',
    },
]

In [10]:
from keras.utils import get_file


def find_weights(weights_collection, model_name, dataset, include_top):
    w = list(filter(lambda x: x['model'] == model_name, weights_collection))
    w = list(filter(lambda x: x['dataset'] == dataset, w))
    w = list(filter(lambda x: x['include_top'] == include_top, w))
    return w


def load_model_weights(weights_collection, model, dataset, classes, include_top):
    weights = find_weights(weights_collection, model.name, dataset, include_top)

    if weights:
        weights = weights[0]

        if include_top and weights['classes'] != classes:
            raise ValueError('If using `weights` and `include_top`'
                             ' as true, `classes` should be {}'.format(weights['classes']))

        weights_path = get_file(weights['name'],
                                weights['url'],
                                cache_subdir='models',
                                md5_hash=weights['md5'])

        model.load_weights(weights_path)

    else:
        raise ValueError('There is no weights for such configuration: ' +
                         'model = {}, dataset = {}, '.format(model.name, dataset) +
                         'classes = {}, include_top = {}.'.format(classes, include_top))

In [11]:
def ResNet34(input_shape, input_tensor=None, weights=None, classes=1000, include_top=True):
    model = build_resnet(input_tensor=input_tensor,
                         input_shape=input_shape,
                         repetitions=(3, 4, 6, 3),
                         classes=classes,
                         include_top=include_top,
                         block_type='basic')
    model.name = 'resnet34'

    if weights:
        load_model_weights(weights_collection, model, weights, classes, include_top)
    return model

In [12]:
# base_model = ResNet34(input_shape=(int(img_width), int(img_height),3), weights='imagenet', classes=1000, include_top = False)

In [13]:
# base_model.save('resnet34.h5')
base_model = load_model(os.path.join(base_dir,'base_models', 'resnet34.h5'))
base_model.summary()

C:\Users\praty\Anaconda3\envs\dlenv\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 299, 299, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_137 (ZeroPadding (None, 305, 305, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 150, 150, 64) 9408        zero_padding2d_137[0][0]         
__________________________________________________________________________________________________
bn0 (Batch

zero_padding2d_145 (ZeroPadding (None, 77, 77, 64)   0           stage2_unit1_relu1[0][0]         
__________________________________________________________________________________________________
stage2_unit1_conv1 (Conv2D)     (None, 38, 38, 128)  73728       zero_padding2d_145[0][0]         
__________________________________________________________________________________________________
stage2_unit1_bn2 (BatchNormaliz (None, 38, 38, 128)  512         stage2_unit1_conv1[0][0]         
__________________________________________________________________________________________________
stage2_unit1_relu2 (Activation) (None, 38, 38, 128)  0           stage2_unit1_bn2[0][0]           
__________________________________________________________________________________________________
zero_padding2d_146 (ZeroPadding (None, 40, 40, 128)  0           stage2_unit1_relu2[0][0]         
__________________________________________________________________________________________________
stage2_uni

stage3_unit1_bn2 (BatchNormaliz (None, 19, 19, 256)  1024        stage3_unit1_conv1[0][0]         
__________________________________________________________________________________________________
stage3_unit1_relu2 (Activation) (None, 19, 19, 256)  0           stage3_unit1_bn2[0][0]           
__________________________________________________________________________________________________
zero_padding2d_154 (ZeroPadding (None, 21, 21, 256)  0           stage3_unit1_relu2[0][0]         
__________________________________________________________________________________________________
stage3_unit1_conv2 (Conv2D)     (None, 19, 19, 256)  589824      zero_padding2d_154[0][0]         
__________________________________________________________________________________________________
stage3_unit1_sc (Conv2D)        (None, 19, 19, 256)  32768       stage3_unit1_relu1[0][0]         
__________________________________________________________________________________________________
add_72 (Ad

zero_padding2d_162 (ZeroPadding (None, 21, 21, 256)  0           stage3_unit5_relu2[0][0]         
__________________________________________________________________________________________________
stage3_unit5_conv2 (Conv2D)     (None, 19, 19, 256)  589824      zero_padding2d_162[0][0]         
__________________________________________________________________________________________________
add_76 (Add)                    (None, 19, 19, 256)  0           stage3_unit5_conv2[0][0]         
                                                                 add_75[0][0]                     
__________________________________________________________________________________________________
stage3_unit6_bn1 (BatchNormaliz (None, 19, 19, 256)  1024        add_76[0][0]                     
__________________________________________________________________________________________________
stage3_unit6_relu1 (Activation) (None, 19, 19, 256)  0           stage3_unit6_bn1[0][0]           
__________

add_80 (Add)                    (None, 10, 10, 512)  0           stage4_unit3_conv2[0][0]         
                                                                 add_79[0][0]                     
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 10, 10, 512)  2048        add_80[0][0]                     
__________________________________________________________________________________________________
relu1 (Activation)              (None, 10, 10, 512)  0           bn1[0][0]                        
Total params: 21,302,473
Trainable params: 21,287,107
Non-trainable params: 15,366
__________________________________________________________________________________________________


In [14]:
# plot_model(base_model, to_file= os.path.join(architecture_plot_save_dir, 'resnet34.png'))

# Define Loss Function

In [15]:
from keras.losses import binary_crossentropy
from keras import backend as K
import tensorflow as tf

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred = K.cast(y_pred, 'float32')
    y_pred_f = K.cast(K.greater(K.flatten(y_pred), 0.5), 'float32')
    intersection = y_true_f * y_pred_f
    score = 2. * K.sum(intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))
    return score

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

def bce_logdice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) - K.log(1. - dice_loss(y_true, y_pred))

def weighted_bce_loss(y_true, y_pred, weight):
    epsilon = 1e-7
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    logit_y_pred = K.log(y_pred / (1. - y_pred))
    loss = weight * (logit_y_pred * (1. - y_true) + 
                     K.log(1. + K.exp(-K.abs(logit_y_pred))) + K.maximum(-logit_y_pred, 0.))
    return K.sum(loss) / K.sum(weight)

def weighted_dice_loss(y_true, y_pred, weight):
    smooth = 1.
    w, m1, m2 = weight, y_true, y_pred
    intersection = (m1 * m2)
    score = (2. * K.sum(w * intersection) + smooth) / (K.sum(w * m1) + K.sum(w * m2) + smooth)
    loss = 1. - K.sum(score)
    return loss

def weighted_bce_dice_loss(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd
    averaged_mask = K.pool2d(
            y_true, pool_size=(50, 50), strides=(1, 1), padding='same', pool_mode='avg')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight = 5. * K.exp(-5. * K.abs(averaged_mask - 0.5))
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    loss = weighted_bce_loss(y_true, y_pred, weight) + dice_loss(y_true, y_pred)
    return loss

In [16]:
def get_iou_vector(A, B):
    batch_size = A.shape[0]
    metric = []
    for batch in range(batch_size):
        t, p = A[batch]>0, B[batch]>0      
        intersection = np.logical_and(t, p)
        union = np.logical_or(t, p)
        iou = (np.sum(intersection > 0) + 1e-10 )/ (np.sum(union > 0) + 1e-10)
        thresholds = np.arange(0.5, 1, 0.05)
        s = []
        for thresh in thresholds:
            s.append(iou > thresh)
        metric.append(np.mean(s))

    return np.mean(metric)

def my_iou_metric(label, pred):
    return tf.py_func(get_iou_vector, [label, pred>0.5], tf.float64)

def my_iou_metric_2(label, pred):
    return tf.py_func(get_iou_vector, [label, pred >0], tf.float64)

In [17]:
for layer in base_model.layers[:-2]:
    layer.trainable = False
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 299, 299, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_137 (ZeroPadding (None, 305, 305, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 150, 150, 64) 9408        zero_padding2d_137[0][0]         
__________________________________________________________________________________________________
bn0 (Batch

zero_padding2d_145 (ZeroPadding (None, 77, 77, 64)   0           stage2_unit1_relu1[0][0]         
__________________________________________________________________________________________________
stage2_unit1_conv1 (Conv2D)     (None, 38, 38, 128)  73728       zero_padding2d_145[0][0]         
__________________________________________________________________________________________________
stage2_unit1_bn2 (BatchNormaliz (None, 38, 38, 128)  512         stage2_unit1_conv1[0][0]         
__________________________________________________________________________________________________
stage2_unit1_relu2 (Activation) (None, 38, 38, 128)  0           stage2_unit1_bn2[0][0]           
__________________________________________________________________________________________________
zero_padding2d_146 (ZeroPadding (None, 40, 40, 128)  0           stage2_unit1_relu2[0][0]         
__________________________________________________________________________________________________
stage2_uni

stage3_unit1_bn2 (BatchNormaliz (None, 19, 19, 256)  1024        stage3_unit1_conv1[0][0]         
__________________________________________________________________________________________________
stage3_unit1_relu2 (Activation) (None, 19, 19, 256)  0           stage3_unit1_bn2[0][0]           
__________________________________________________________________________________________________
zero_padding2d_154 (ZeroPadding (None, 21, 21, 256)  0           stage3_unit1_relu2[0][0]         
__________________________________________________________________________________________________
stage3_unit1_conv2 (Conv2D)     (None, 19, 19, 256)  589824      zero_padding2d_154[0][0]         
__________________________________________________________________________________________________
stage3_unit1_sc (Conv2D)        (None, 19, 19, 256)  32768       stage3_unit1_relu1[0][0]         
__________________________________________________________________________________________________
add_72 (Ad

zero_padding2d_162 (ZeroPadding (None, 21, 21, 256)  0           stage3_unit5_relu2[0][0]         
__________________________________________________________________________________________________
stage3_unit5_conv2 (Conv2D)     (None, 19, 19, 256)  589824      zero_padding2d_162[0][0]         
__________________________________________________________________________________________________
add_76 (Add)                    (None, 19, 19, 256)  0           stage3_unit5_conv2[0][0]         
                                                                 add_75[0][0]                     
__________________________________________________________________________________________________
stage3_unit6_bn1 (BatchNormaliz (None, 19, 19, 256)  1024        add_76[0][0]                     
__________________________________________________________________________________________________
stage3_unit6_relu1 (Activation) (None, 19, 19, 256)  0           stage3_unit6_bn1[0][0]           
__________

add_80 (Add)                    (None, 10, 10, 512)  0           stage4_unit3_conv2[0][0]         
                                                                 add_79[0][0]                     
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 10, 10, 512)  2048        add_80[0][0]                     
__________________________________________________________________________________________________
relu1 (Activation)              (None, 10, 10, 512)  0           bn1[0][0]                        
Total params: 21,302,473
Trainable params: 1,024
Non-trainable params: 21,301,449
__________________________________________________________________________________________________


In [18]:
x = base_model.output
x = Flatten()(x)
predictions = Dense(4, activation='relu')(x)

In [19]:
model = Model(input=base_model.input, output=predictions)
model.summary()

C:\Users\praty\Anaconda3\envs\dlenv\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("da..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 299, 299, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_137 (ZeroPadding (None, 305, 305, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 150, 150, 64) 9408        zero_padding2d_137[0][0]         
__________________________________________________________________________________________________
bn0 (Batch

zero_padding2d_145 (ZeroPadding (None, 77, 77, 64)   0           stage2_unit1_relu1[0][0]         
__________________________________________________________________________________________________
stage2_unit1_conv1 (Conv2D)     (None, 38, 38, 128)  73728       zero_padding2d_145[0][0]         
__________________________________________________________________________________________________
stage2_unit1_bn2 (BatchNormaliz (None, 38, 38, 128)  512         stage2_unit1_conv1[0][0]         
__________________________________________________________________________________________________
stage2_unit1_relu2 (Activation) (None, 38, 38, 128)  0           stage2_unit1_bn2[0][0]           
__________________________________________________________________________________________________
zero_padding2d_146 (ZeroPadding (None, 40, 40, 128)  0           stage2_unit1_relu2[0][0]         
__________________________________________________________________________________________________
stage2_uni

stage3_unit1_bn2 (BatchNormaliz (None, 19, 19, 256)  1024        stage3_unit1_conv1[0][0]         
__________________________________________________________________________________________________
stage3_unit1_relu2 (Activation) (None, 19, 19, 256)  0           stage3_unit1_bn2[0][0]           
__________________________________________________________________________________________________
zero_padding2d_154 (ZeroPadding (None, 21, 21, 256)  0           stage3_unit1_relu2[0][0]         
__________________________________________________________________________________________________
stage3_unit1_conv2 (Conv2D)     (None, 19, 19, 256)  589824      zero_padding2d_154[0][0]         
__________________________________________________________________________________________________
stage3_unit1_sc (Conv2D)        (None, 19, 19, 256)  32768       stage3_unit1_relu1[0][0]         
__________________________________________________________________________________________________
add_72 (Ad

zero_padding2d_162 (ZeroPadding (None, 21, 21, 256)  0           stage3_unit5_relu2[0][0]         
__________________________________________________________________________________________________
stage3_unit5_conv2 (Conv2D)     (None, 19, 19, 256)  589824      zero_padding2d_162[0][0]         
__________________________________________________________________________________________________
add_76 (Add)                    (None, 19, 19, 256)  0           stage3_unit5_conv2[0][0]         
                                                                 add_75[0][0]                     
__________________________________________________________________________________________________
stage3_unit6_bn1 (BatchNormaliz (None, 19, 19, 256)  1024        add_76[0][0]                     
__________________________________________________________________________________________________
stage3_unit6_relu1 (Activation) (None, 19, 19, 256)  0           stage3_unit6_bn1[0][0]           
__________

add_80 (Add)                    (None, 10, 10, 512)  0           stage4_unit3_conv2[0][0]         
                                                                 add_79[0][0]                     
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 10, 10, 512)  2048        add_80[0][0]                     
__________________________________________________________________________________________________
relu1 (Activation)              (None, 10, 10, 512)  0           bn1[0][0]                        
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 51200)        0           relu1[0][0]                      
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 4)            204804      flatten_1[0][0]                  
Total para

In [20]:
def mIoU(labels,predictions):
    pred_x1=predictions[:,0]#*img_width
    pred_x2=predictions[:,1]#*img_width
    pred_y1=predictions[:,2]#*img_height
    pred_y2=predictions[:,3]#*img_height
    label_x1=labels[:,0]#*img_width
    label_x2=labels[:,1]#*img_width
    label_y1=labels[:,2]#*img_height
    label_y2=labels[:,3]#*img_height
    int_x1 = tf.maximum(pred_x1, label_x1)
    int_x2 = tf.minimum(pred_x2, label_x2)
    int_y1 = tf.maximum(pred_y1, label_y1)
    int_y2 = tf.minimum(pred_y2, label_y2)
    tensor_type = pred_x1.dtype
    pred_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), pred_x2 - pred_x1), tf.maximum(tf.cast(0.0, tensor_type), pred_y2 - pred_y1))
    label_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), label_x2 - label_x1), tf.maximum(tf.cast(0.0, tensor_type), label_y2 - label_y1))
    int_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), int_x2 - int_x1), tf.maximum(tf.cast(0.0, tensor_type), int_y2 - int_y1))
    union_area = pred_area + label_area - int_area
    IoU = int_area / union_area
    return tf.reduce_mean(IoU)

In [21]:
model.compile(loss ="mse", optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), metrics=[mIoU])

# Training

In [22]:
checkpoint = ModelCheckpoint(chkpt_dir, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=40, verbose=1, mode='auto')

In [23]:
train_df = pd.read_csv(os.path.join(base_dir, 'augument_data', 'augmented_train_data.csv'))
val_df = pd.read_csv(os.path.join(os.path.join(base_dir, 'augument_data', 'augmented_val_data.csv')))

In [24]:
l = []
for im in val_df.image_name:
    img = (cv2.imread(os.path.join(train_path, im))) ### INTRIDUCING NORMALIZATION.
    img = cv2.resize(img, (int(img_width), int(img_height)))
    img = (img / 255.).astype(np.float32)
    l.append(np.expand_dims(img, axis = 0))
x_val = np.vstack(l)
print("x_val done.")
y_val = val_df.iloc[:,1:].values.astype(np.float32)
y_val[:, 0] = y_val[:, 0] # / img_width
y_val[:, 1] = y_val[:, 1] # / img_width
y_val[:, 2] = y_val[:, 2] # / img_height
y_val[:, 3] = y_val[:, 3] # / img_height
print(y_val[0])
print(y_val.dtype)
print(x_val.shape)


x_val done.
[ 78. 578.  78. 317.]
float32
(2000, 299, 299, 3)


In [25]:
print(val_df.image_name[0])
model.load_weights(chkpt_dir)

[h_v]1475157534672DSC07249.png


In [26]:
ram_batch_size = 1000
total_number_images = 68000
print(total_number_images // ram_batch_size)

68


In [27]:
def show_img_with_rect(img, lab):
    x1 = int(lab[0]/2.1404)
    x2 = int(lab[1]/2.1404)
    y1 = int(lab[2]/1.61)
    y2 = int(lab[3]/1.61) #
    cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),3)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [28]:
ctr = 0

for epoch in range(epochs):
    print(epoch)
    print("--------------------------------------------------------------")
    for i in range(total_number_images // ram_batch_size):
        print('batch {}/67 (epoch {}):'.format(i, epoch))
        l = []
        x_train = np.load(os.path.join(npy_img_dir, str(i) + '.npy'))
        y_train = train_df[i * ram_batch_size: (i + 1) * ram_batch_size].iloc[:,1:].values
        
        model.fit(x=x_train, y=y_train, batch_size=32, epochs=1, verbose=1, callbacks=[checkpoint], validation_split=0.0, validation_data=(x_val, y_val), shuffle=True)

0
--------------------------------------------------------------
batch 0/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - ETA: 53s - loss: 438.6616 - mIoU: 0.809 - ETA: 27s - loss: 538.9256 - mIoU: 0.782 - ETA: 19s - loss: 523.3218 - mIoU: 0.785 - ETA: 14s - loss: 537.7081 - mIoU: 0.797 - ETA: 11s - loss: 490.1202 - mIoU: 0.806 - ETA: 9s - loss: 484.3977 - mIoU: 0.809 - ETA: 8s - loss: 453.9431 - mIoU: 0.81 - ETA: 7s - loss: 461.8717 - mIoU: 0.81 - ETA: 6s - loss: 441.4550 - mIoU: 0.82 - ETA: 6s - loss: 427.5688 - mIoU: 0.82 - ETA: 5s - loss: 446.9827 - mIoU: 0.82 - ETA: 4s - loss: 454.7947 - mIoU: 0.82 - ETA: 4s - loss: 526.1910 - mIoU: 0.82 - ETA: 4s - loss: 513.5839 - mIoU: 0.82 - ETA: 3s - loss: 501.5778 - mIoU: 0.83 - ETA: 3s - loss: 513.8896 - mIoU: 0.83 - ETA: 3s - loss: 507.4350 - mIoU: 0.83 - ETA: 2s - loss: 525.0465 - mIoU: 0.83 - ETA: 2s - loss: 519.3209 - mIoU: 0.83 - ETA: 2s - loss: 508.9351 - mIoU: 0.83 -

1000/1000 [==============================] - ETA: 3s - loss: 523.9803 - mIoU: 0.80 - ETA: 3s - loss: 676.4062 - mIoU: 0.79 - ETA: 3s - loss: 705.2831 - mIoU: 0.80 - ETA: 3s - loss: 611.0825 - mIoU: 0.81 - ETA: 3s - loss: 639.9784 - mIoU: 0.82 - ETA: 2s - loss: 636.3877 - mIoU: 0.82 - ETA: 2s - loss: 643.1572 - mIoU: 0.82 - ETA: 2s - loss: 616.0928 - mIoU: 0.82 - ETA: 2s - loss: 570.4004 - mIoU: 0.82 - ETA: 2s - loss: 544.2535 - mIoU: 0.83 - ETA: 2s - loss: 528.0220 - mIoU: 0.83 - ETA: 2s - loss: 515.1743 - mIoU: 0.83 - ETA: 2s - loss: 531.8983 - mIoU: 0.83 - ETA: 1s - loss: 517.1017 - mIoU: 0.83 - ETA: 1s - loss: 523.4374 - mIoU: 0.83 - ETA: 1s - loss: 522.5325 - mIoU: 0.83 - ETA: 1s - loss: 566.0279 - mIoU: 0.83 - ETA: 1s - loss: 565.0862 - mIoU: 0.83 - ETA: 1s - loss: 568.1040 - mIoU: 0.83 - ETA: 1s - loss: 555.6978 - mIoU: 0.83 - ETA: 1s - loss: 552.8600 - mIoU: 0.83 - ETA: 1s - loss: 626.2519 - mIoU: 0.83 - ETA: 0s - loss: 616.3892 - mIoU: 0.83 - ETA: 0s - loss: 600.8985 - mIoU: 0.

1000/1000 [==============================] - ETA: 3s - loss: 431.1179 - mIoU: 0.85 - ETA: 3s - loss: 373.9911 - mIoU: 0.85 - ETA: 3s - loss: 845.5941 - mIoU: 0.84 - ETA: 3s - loss: 727.5274 - mIoU: 0.83 - ETA: 3s - loss: 668.8345 - mIoU: 0.83 - ETA: 2s - loss: 657.1608 - mIoU: 0.83 - ETA: 2s - loss: 611.6884 - mIoU: 0.83 - ETA: 2s - loss: 607.9289 - mIoU: 0.83 - ETA: 2s - loss: 582.3941 - mIoU: 0.83 - ETA: 2s - loss: 611.6740 - mIoU: 0.83 - ETA: 2s - loss: 592.3530 - mIoU: 0.83 - ETA: 2s - loss: 559.8122 - mIoU: 0.83 - ETA: 2s - loss: 568.7657 - mIoU: 0.83 - ETA: 1s - loss: 545.4647 - mIoU: 0.84 - ETA: 1s - loss: 533.4979 - mIoU: 0.84 - ETA: 1s - loss: 625.7349 - mIoU: 0.84 - ETA: 1s - loss: 619.1323 - mIoU: 0.83 - ETA: 1s - loss: 603.5948 - mIoU: 0.83 - ETA: 1s - loss: 605.8929 - mIoU: 0.84 - ETA: 1s - loss: 595.9602 - mIoU: 0.84 - ETA: 1s - loss: 587.2136 - mIoU: 0.84 - ETA: 1s - loss: 590.6700 - mIoU: 0.84 - ETA: 0s - loss: 581.7933 - mIoU: 0.84 - ETA: 0s - loss: 565.6899 - mIoU: 0.

1000/1000 [==============================] - ETA: 3s - loss: 274.3514 - mIoU: 0.86 - ETA: 3s - loss: 477.8129 - mIoU: 0.82 - ETA: 3s - loss: 411.5954 - mIoU: 0.83 - ETA: 3s - loss: 658.9845 - mIoU: 0.83 - ETA: 3s - loss: 613.7859 - mIoU: 0.84 - ETA: 2s - loss: 568.2024 - mIoU: 0.84 - ETA: 2s - loss: 568.7177 - mIoU: 0.83 - ETA: 2s - loss: 526.8023 - mIoU: 0.84 - ETA: 2s - loss: 496.9509 - mIoU: 0.84 - ETA: 2s - loss: 470.7015 - mIoU: 0.84 - ETA: 2s - loss: 460.8688 - mIoU: 0.84 - ETA: 2s - loss: 447.7245 - mIoU: 0.84 - ETA: 2s - loss: 457.3162 - mIoU: 0.84 - ETA: 1s - loss: 466.7188 - mIoU: 0.84 - ETA: 1s - loss: 455.2287 - mIoU: 0.84 - ETA: 1s - loss: 455.2542 - mIoU: 0.84 - ETA: 1s - loss: 461.4148 - mIoU: 0.84 - ETA: 1s - loss: 464.2786 - mIoU: 0.84 - ETA: 1s - loss: 470.2635 - mIoU: 0.84 - ETA: 1s - loss: 472.0576 - mIoU: 0.83 - ETA: 1s - loss: 469.9289 - mIoU: 0.83 - ETA: 1s - loss: 469.4098 - mIoU: 0.83 - ETA: 0s - loss: 482.5443 - mIoU: 0.83 - ETA: 0s - loss: 476.6665 - mIoU: 0.

KeyboardInterrupt: 

In [ ]:
model.save('resnet34_trained.h5')

In [ ]:
for i in range(0, 2000, 100):
    pred = model.predict(np.expand_dims(x_val[i], axis=0))
    a1 = int(pred[0][0]/rx)
    a2 = int(pred[0][1]/rx)
    b1 = int(pred[0][2]/ry)
    b2 = int(pred[0][3]/ry)
    test_img = x_val[i]
    print(test_img.shape)
    plt.figure(i)
    cv2.rectangle(test_img,(a1,b1),(a2,b2),(0,255,0),3)
    plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))

In [ ]:
test_img = x_val[7]
test_img = np.expand_dims(test_img, axis=0)
pred = model.predict(test_img)
print(pred.shape)
a1 = int(pred[0][0]/rx)
a2 = int(pred[0][1]/rx)
b1 = int(pred[0][2]/ry)
b2 = int(pred[0][3]/ry)
test_img = test_img[0]
cv2.rectangle(test_img,(a1,b1),(a2,b2),(0,255,0),3)
plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))

In [ ]:
model.load_weights(chkpt_dir)
LABEL = y_val
PRED = model.predict(x_val)
with tf.Session() as sess:
    val = sess.run(mIoU(LABEL, PRED))
    print(val)

In [29]:
import fastai